In [1]:
%matplotlib inline

import re
import numpy as np
import pandas as pd
import catboost as cb
import matplotlib.pyplot as plt

from fuzzywuzzy import fuzz
from tqdm import tqdm_notebook
from txt_feat import get_features
from sklearn.cluster import KMeans
from utils import get_disease_class
from lstm import KTokenizer, get_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

/home/s_belyaev/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN_PATH = '~/.kaggle/competitions/ai-hack-2018-spb-robomed/train_data.csv'
TEST_PATH = '~/.kaggle/competitions/ai-hack-2018-spb-robomed/test_data.csv'
SUBMISSION_PATH = 'submission.csv'
MODEL_SAVE_PATH = 'model.pkl'

In [3]:
def initial_pre(df):
    df = df.copy()
    del df['ID']
    rename_map = {
        'Код_диагноза': 'diag_code',
        'Диагноз': 'diag',
        'Возраст': 'age',
        'Пол': 'sex',
        'Общее состояние': 'state',
        'аллергия': 'allergies',
        'Анамнез заболевания': 'anamnesis',
        'Внешний осмотр': 'checkup',
        'Revisit': 'revisit'
    }
    return df.rename(columns=rename_map)

In [4]:
df = initial_pre(pd.read_csv(TRAIN_PATH))
len(df)

283086

In [6]:
df[df.revisit == 0].describe(include='all')

,diag_code,diag,age,sex,state,allergies,anamnesis,checkup,revisit
count,48510,48510,48510.000000,48510.000000,3161,2943,43030,557,48510.0
unique,2328,2312,NaN,NaN,470,619,34702,87,NaN
top,M42.1,Остеохондроз позвоночника у взрослых,NaN,NaN,Общее самочувствие удовлетворительное. Кожные ...,не отягощен,Без отрицательной динамики в неврологическом с...,удовлетворительное.,NaN
freq,2576,2576,NaN,NaN,735,1255,618,300,NaN
mean,NaN,NaN,43.166770,1.613296,NaN,NaN,NaN,NaN,0.0
std,NaN,NaN,37.410713,0.487000,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,0.0
25%,NaN,NaN,31.000000,1.000000,NaN,NaN,NaN,NaN,0.0
50%,NaN,NaN,39.000000,2.000000,NaN,NaN,NaN,NaN,0.0
75%,NaN,NaN,54.000000,2.000000,NaN,NaN,NaN,NaN,0.0


In [7]:
df[df.revisit == 1].describe(include='all')

,diag_code,diag,age,sex,state,allergies,anamnesis,checkup,revisit
count,234576,234576,234576.000000,234576.000000,38535,37965,160626,12738,234576.0
unique,3113,3077,NaN,NaN,2800,4900,121844,767,NaN
top,N77.1*,"Вагинит, вульвит и вульвовагинит при инфекцион...",NaN,NaN,Общее самочувствие удовлетворительное. Кожные ...,не отягощен,консультация по проведенному обследованию и ко...,удовлетворительное.,NaN
freq,10721,10721,NaN,NaN,8225,16106,2464,6322,NaN
mean,NaN,NaN,41.392939,1.728003,NaN,NaN,NaN,NaN,1.0
std,NaN,NaN,16.431538,0.444989,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,1.0
25%,NaN,NaN,31.000000,1.000000,NaN,NaN,NaN,NaN,1.0
50%,NaN,NaN,38.000000,2.000000,NaN,NaN,NaN,NaN,1.0
75%,NaN,NaN,50.000000,2.000000,NaN,NaN,NaN,NaN,1.0
